In [16]:
import pygame
import sys
import random
from collections import deque

# Definicje kolorów
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
YELLOW = (255, 255, 0)

# Definicje planszy
MAP = [
    list("############################"),
    list("#............##............#"),
    list("#.####.#####.##.#####.####.#"),
    list("#o####.#####.##.#####.####o#"),
    list("#.####.#####.##.#####.####.#"),
    list("#..........................#"),
    list("#.####.##.########.##.####.#"),
    list("#.####.##.########.##.####.#"),
    list("#......##....##....##......#"),
    list("######.##### ## #####.######"),
    list("     #.##### ## #####.#     "),
    list("     #.##          ##.#     "),
    list("     #.## ##    ## ##.#     "),
    list("######.## ##    ## ##.######"),
    list("      .   ##    ##   .      "),
    list("######.#####    #####.######"),
    list("     #.#####    #####.#     "),
    list("     #.##          ##.#     "),
    list("     #.## ######## ##.#     "),
    list("######.## ######## ##.######"),
    list("#............##............#"),
    list("#.####.#####.##.#####.####.#"),
    list("#o..##.#...................o#"),
    list("###.##.#.########.##########"),
    list("###.##.#.########.##########"),
    list("#......##....##............#"),
    list("#.##########.##.##########.#"),
    list("#.##########.##.##########.#"),
    list("#..........................#"),
    list("############################")
]

# Parametry planszy
BLOCK_SIZE = 20
MAP_WIDTH = len(MAP[0])
MAP_HEIGHT = len(MAP)
WINDOW_WIDTH = MAP_WIDTH * BLOCK_SIZE
WINDOW_HEIGHT = MAP_HEIGHT * BLOCK_SIZE

# Inicjalizacja Pygame
pygame.init()

# Ustawienie okna gry
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Pacman")

# Pozycja Pacmana
pacman_x = 1
pacman_y = 1

# Kierunek ruchu Pacmana
direction = None

# Pozycje duszków
ghosts = [
    {"position": (12, 13), "target": None, "path": []},
    {"position": (15, 13), "target": None, "path": []},
    {"position": (12, 15), "target": None, "path": []},
    {"position": (15, 15), "target": None, "path": []}
]

# Licznik punktów
points = 0

# Liczniki iteracji
pacman_move_counter = 0
ghost_move_counter = 0

# Progi iteracji
PACMAN_MOVE_DELAY = 30  # Im wyższa wartość, tym wolniejszy ruch Pacmana
GHOST_MOVE_DELAY = 45  # Im wyższa wartość, tym wolniejszy ruch duszków

# Stan gry
game_state = "playing"

# Funkcja rysująca planszę
def draw_map():
    global points
    for y in range(MAP_HEIGHT):
        for x in range(MAP_WIDTH):
            if MAP[y][x] == "#":
                pygame.draw.rect(window, BLUE, (x * BLOCK_SIZE, y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))
            elif MAP[y][x] == ".":
                pygame.draw.circle(window, WHITE, (x * BLOCK_SIZE + BLOCK_SIZE // 2, y * BLOCK_SIZE + BLOCK_SIZE // 2), 3)
                if pacman_x == x and pacman_y == y:
                    MAP[y][x] = " "  # Usuń punkt z planszy
                    points += 1
            elif MAP[y][x] == "o":
                pygame.draw.circle(window, YELLOW, (x * BLOCK_SIZE + BLOCK_SIZE // 2, y * BLOCK_SIZE + BLOCK_SIZE // 2), BLOCK_SIZE // 4)
                if pacman_x == x and pacman_y == y:
                    MAP[y][x] = " "  # Usuń punkt z planszy
                    points += 10

# Funkcja rysująca Pacmana
def draw_pacman(x, y):
    pygame.draw.rect(window, YELLOW, (x * BLOCK_SIZE, y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))

# Funkcja rysująca duszki
def draw_ghosts():
    for ghost in ghosts:
        x, y = ghost["position"]
        pygame.draw.rect(window, WHITE, (x * BLOCK_SIZE, y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))

# Funkcja wyświetlająca wynik
def draw_score():
    font = pygame.font.SysFont(None, 30)
    score_text = font.render("Wynik: " + str(points), True, WHITE)
    window.blit(score_text, (10, WINDOW_HEIGHT - 20))

# Funkcja BFS do znalezienia najkrótszej ścieżki
def bfs(start, end):
    queue = deque([(start, [])])
    visited = set()
    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == end:
            return path
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < MAP_WIDTH and 0 <= ny < MAP_HEIGHT and MAP[ny][nx] != "#" and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append(((nx, ny), path + [(nx, ny)]))
    return None

# Funkcja sprawdzająca, czy Pacman jest w zasięgu duszka
def is_pacman_near(ghost_x, ghost_y, range):
    return abs(ghost_x - pacman_x) <= range and abs(ghost_y - pacman_y) <= range

# Funkcja ruchu duszków
def move_ghosts():
    global ghosts
    for ghost in ghosts:
        if is_pacman_near(ghost["position"][0], ghost["position"][1], 4):
            ghost["target"] = (pacman_x, pacman_y)
            ghost["path"] = bfs(ghost["position"], ghost["target"])
        
        if ghost["path"]:
            next_pos = ghost["path"].pop(0)
            ghost["position"] = next_pos
        else:
            possible_moves = []
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_x, new_y = ghost["position"][0] + dx, ghost["position"][1] + dy
                if 0 <= new_x < MAP_WIDTH and 0 <= new_y < MAP_HEIGHT and MAP[new_y][new_x] != "#":
                    possible_moves.append((new_x, new_y))
            if possible_moves:
                ghost["position"] = random.choice(possible_moves)

# Funkcja sprawdzająca kolizje
def check_collision():
    for ghost in ghosts:
        if pacman_x == ghost["position"][0] and pacman_y == ghost["position"][1]:
            return True
    return False

# Funkcja sprawdzająca wygraną
def check_win():
    for row in MAP:
        if "." in row or "o" in row:
            return False
    return True

# Funkcja resetująca grę
def reset_game():
    global pacman_x, pacman_y, points, ghosts, ghost_move_counter, game_state, MAP, direction, pacman_move_counter
    pacman_x, pacman_y = 1, 1
    points = 0
    ghost_move_counter = 0
    pacman_move_counter = 0
    game_state = "playing"
    direction = None
    ghosts = [
        {"position": (12, 13), "target": None, "path": []},
        {"position": (15, 13), "target": None, "path": []},
        {"position": (12, 15), "target": None, "path": []},
        {"position": (15, 15), "target": None, "path": []}
    ]
    MAP = [
        list("############################"),
        list("#............##............#"),
        list("#.####.#####.##.#####.####.#"),
        list("#o####.#####.##.#####.####o#"),
        list("#.####.#####.##.#####.####.#"),
        list("#..........................#"),
        list("#.####.##.########.##.####.#"),
        list("#.####.##.########.##.####.#"),
        list("#......##....##....##......#"),
        list("######.##### ## #####.######"),
        list("     #.##### ## #####.#     "),
        list("     #.##          ##.#     "),
        list("     #.## ##    ## ##.#     "),
        list("######.## ##    ## ##.######"),
        list("      .   ##    ##   .      "),
        list("######.#####    #####.######"),
        list("     #.#####    #####.#     "),
        list("     #.##          ##.#     "),
        list("     #.## ######## ##.#     "),
        list("######.## ######## ##.######"),
        list("#............##............#"),
        list("#.####.#####.##.#####.####.#"),
        list("#o..##.#...................o#"),
        list("###.##.#.########.##########"),
        list("###.##.#.########.##########"),
        list("#......##....##............#"),
        list("#.##########.##.##########.#"),
        list("#.##########.##.##########.#"),
        list("#..........................#"),
        list("############################")
    ]

# Funkcja rysująca panel końcowy
def draw_game_over_panel():
    font = pygame.font.SysFont(None, 50)
    game_over_text = font.render("Game Over", True, WHITE)
    try_again_text = font.render("Try Again", True, WHITE)

    game_over_rect = game_over_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 - 50))
    try_again_rect = try_again_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 50))

    window.blit(game_over_text, game_over_rect)
    window.blit(try_again_text, try_again_rect)

    return try_again_rect

# Funkcja rysująca panel wygranej
def draw_win_panel():
    font = pygame.font.SysFont(None, 50)
    win_text = font.render("You Win!", True, WHITE)
    play_again_text = font.render("Play Again", True, WHITE)

    win_rect = win_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 - 50))
    play_again_rect = play_again_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 50))

    window.blit(win_text, win_rect)
    window.blit(play_again_text, play_again_rect)

    return play_again_rect

# Główna pętla gry
def main():
    global pacman_x, pacman_y, points, ghosts, ghost_move_counter, game_state, direction, pacman_move_counter

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN and game_state in ["game_over", "win"]:
                mouse_pos = event.pos
                if game_state == "game_over":
                    try_again_rect = draw_game_over_panel()
                    if try_again_rect.collidepoint(mouse_pos):
                        reset_game()
                elif game_state == "win":
                    play_again_rect = draw_win_panel()
                    if play_again_rect.collidepoint(mouse_pos):
                        reset_game()
            elif event.type == pygame.KEYDOWN and game_state == "playing":
                if event.key == pygame.K_LEFT:
                    direction = "left"
                elif event.key == pygame.K_RIGHT:
                    direction = "right"
                elif event.key == pygame.K_UP:
                    direction = "up"
                elif event.key == pygame.K_DOWN:
                    direction = "down"
            elif event.type == pygame.KEYUP:
                if event.key in (pygame.K_LEFT, pygame.K_RIGHT, pygame.K_UP, pygame.K_DOWN):
                    direction = None

        if game_state == "playing":
            pacman_move_counter += 1
            if pacman_move_counter >= PACMAN_MOVE_DELAY:
                if direction == "left" and MAP[pacman_y][pacman_x - 1] != "#":
                    pacman_x -= 1
                elif direction == "right" and MAP[pacman_y][pacman_x + 1] != "#":
                    pacman_x += 1
                elif direction == "up" and MAP[pacman_y - 1][pacman_x] != "#":
                    pacman_y -= 1
                elif direction == "down" and MAP[pacman_y + 1][pacman_x] != "#":
                    pacman_y += 1
                pacman_move_counter = 0

            # Przesuwanie duszków co określoną liczbę iteracji
            ghost_move_counter += 1
            if ghost_move_counter >= GHOST_MOVE_DELAY:
                move_ghosts()
                ghost_move_counter = 0

            # Sprawdzanie kolizji
            if check_collision():
                game_state = "game_over"

            # Sprawdzanie wygranej
            if check_win():
                game_state = "win"

            window.fill(BLACK)
            draw_map()
            draw_pacman(pacman_x, pacman_y)
            draw_ghosts()
            draw_score()
        elif game_state == "game_over":
            draw_game_over_panel()
        elif game_state == "win":
            draw_win_panel()

        pygame.display.update()

if __name__ == "__main__":
    main()